In [ ]:
# Treasury Forecasting algo version #2 determining the best investment windows
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

In [ ]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1', sql=False):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        if sql:
            cursor.execute(sql)
        else:
            query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
            cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [ ]:
# STEP 1: Get the asset classes combined with their parent class ID
table_name = 'AssetClass'
sql = """SELECT a.`ID`, a.`Title`, a.`Group`, a.`Issuer`, a.`PercentMax`,
CASE WHEN p.`Title` IS NULL THEN a.`Title` ELSE p.`Title` END AS `AssetClassCombined`
FROM `AssetClass` a
LEFT JOIN ( SELECT ac.`ID`, ac.`Title`, ac.`Group`, ac.`PercentMax` FROM `AssetClass` ac
WHERE AssetClassParentID = 0 ) p ON p.ID = a.AssetClassParentID
WHERE AssetClassParentID = 0 AND a.Title != 'Not Assigned'
"""
asset_classes = fetch_data(table_name, '',1,sql)
asset_classes

In [ ]:
# Load the windows data
windows = pd.read_pickle('invest_windows.pkl')
windows.info()

In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime

# Assuming we have these dataframes already loaded:
# - windows: contains the investment window data
# - asset_classes: contains the asset class ID and Title mapping

# Function to insert windows data into MySQL table
def insert_windows_data(windows_df, asset_classes_df, conn):
    """
    Insert data from windows dataframe into the InvestmentWindow MySQL table,
    looking up AssetClassID from asset_classes dataframe.

    Args:
        windows_df: DataFrame containing window data
        asset_classes_df: DataFrame containing asset class mappings
        conn: MySQL connection object
    """
    # Create a cursor
    cursor = conn.cursor()

    # Create a mapping dictionary from asset class title to ID
    asset_class_mapping = dict(zip(asset_classes_df['Title'], asset_classes_df['ID']))

    # Get current datetime for LastEdited and Created fields
    current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Prepare insert statement
    insert_query = """
    INSERT INTO InvestmentWindow
    (ClassName, LastEdited, Created, FromDate, EndDate, Available, Days, AssetClassID)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Counter for successful and failed inserts
    successful = 0
    failed = 0

    # Process each row in the windows dataframe
    for _, row in windows_df.iterrows():
        try:
            # Look up AssetClassID from the asset_classes dataframe
            asset_class_title = row['Asset Class']
            asset_class_id = asset_class_mapping.get(asset_class_title)

            if asset_class_id is None:
                print(f"Warning: Asset class '{asset_class_title}' not found in asset_classes dataframe")
                failed += 1
                continue

            # Prepare data for insert
            data = (
                'App\\Model\\InvestmentWindow',  # ClassName
                current_datetime,                # LastEdited
                current_datetime,                # Created
                row['StartDate'].strftime('%Y-%m-%d'),  # FromDate
                row['EndDate'].strftime('%Y-%m-%d'),    # EndDate
                float(row['LowPointBalance']),   # Available
                int(row['TimeSpanDays']),        # Days
                int(asset_class_id)              # AssetClassID
            )

            # Execute insert
            cursor.execute(insert_query, data)
            successful += 1

        except Exception as e:
            print(f"Error inserting row: {e}")
            failed += 1

    # Commit the transaction
    conn.commit()

    # Close cursor
    cursor.close()

    print(f"Data import complete. {successful} rows inserted successfully, {failed} rows failed.")

# Example usage
if __name__ == "__main__":
    # This is just an example; in a real scenario you would use your actual dataframes and connection
    # insert_windows_data(windows, asset_classes, conn)

    # For demonstration, here's how you would call the function with the given information:
    """
    # Assuming these are already defined:
    # - windows: the dataframe with investment window data
    # - asset_classes: the dataframe with asset class IDs and titles
    # - conn: the MySQL connection created with mysql.connector.connect(**DB_CONFIG)

    insert_windows_data(windows, asset_classes, conn)
    """

In [ ]:
def truncate_investment_window_table(conn):
    """
    Truncate the InvestmentWindow table to remove all existing data.

    Args:
        conn: MySQL connection object
    """
    cursor = conn.cursor()
    try:
        cursor.execute("TRUNCATE TABLE InvestmentWindow")
        print("InvestmentWindow table truncated successfully")
    except Exception as e:
        print(f"Error truncating InvestmentWindow table: {e}")
    finally:
        cursor.close()

In [ ]:
conn = conn = mysql.connector.connect(**DB_CONFIG)
truncate_investment_window_table(conn)
insert_windows_data(windows, asset_classes, conn)
